In [1]:
'''#carparkID is a list of strings, date is in the format dd/mm/yyyy , predhr is an int of hour in 24h format
def predictCarparkSlots(carparkIDs, predDate, predhr): 
    #Pandas, Json and formatting stuff
    import pandas as pd
    from datetime import datetime
    import numpy as np

    #For file iteration
    import os
    
    #For Graphing
    import seaborn as sb
    import matplotlib.pyplot as plt 
    sb.set()
    
    #linreg
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures

    counter = 0
    #Create Dictionary to store all data
    #Dict key = Date
    carparkInfoDict = {}

    for filename in os.listdir("./Data/"):
        if filename.endswith(".csv"): 
            #print(os.path.join("./Data/", filename))
            path = os.path.join("./Data/", filename)
            formatDate = filename.replace("_carpark_availability.csv", "")
            formatDate = formatDate.replace("_", "/")
            carparkInfoDict[formatDate] = pd.read_csv(path)
            continue
        else:
            continue
        
    for i in carparkInfoDict:
        #carparkInfoDict[i].columns = ["Time", "Date", "CarParkID","Area","Development", "Location", "AvailableLots", "LotType", "Agency"]
        carparkInfoDict[i]['Time'] = pd.to_timedelta(carparkInfoDict[i]['Time'])
        carparkInfoDict[i]['Time'] = carparkInfoDict[i]['Time'].dt.total_seconds().div(60).astype(int)
        #carparkInfoDict[i]["DateTime"] = pd.to_datetime(carparkInfoDict[i]["Date"] + ' ' + carparkInfoDict[i]["Time"])
        #carparkInfoDict[i].set_index("DateTime")
        #         print(i)
        #         print(carparkInfoDict[i].head())
        
    print("It is predicted that there are:")
    for carparkID in carparkIDs:
        #Create a dataframe for the selected carpark id
        lotinfo = pd.DataFrame(columns = ["Time", "Date", "CarParkID","Area","Development", "Location", "AvailableLots", "LotType", "Agency"])
        for i in carparkInfoDict:
            #for weekdays
            if (datetime.strptime(i, '%d/%m/%Y').weekday() < 5 and datetime.strptime(predDate, '%d/%m/%Y').weekday() < 5):
                #print("weekday "+ i)
                lotinfo = lotinfo.append(carparkInfoDict[i].query('CarParkID == "%s" and LotType == "C"' %carparkID))
                #lotinfoWD['Time'] = lotinfoWD['Time'].str[:2]
            elif (datetime.strptime(i, '%d/%m/%Y').weekday() >= 5 and datetime.strptime(predDate, '%d/%m/%Y').weekday() >= 5):
                #print("weekend "+ i)
                lotinfo = lotinfo.append(carparkInfoDict[i].query('CarParkID == "%s" and LotType == "C"' %carparkID))
                #lotinfoWE['Time'] = lotinfoWE['Time'].str[:2]
            else: 
                pass


        lotinfo = lotinfo.sort_values(by=['Time'])
        lotinfo = lotinfo.reset_index()

        #Visualise data (Optional)
        f = plt.figure(figsize=(30, 15))
        plt.scatter(lotinfo['Time']/60, lotinfo['AvailableLots'])
        #f= plt.yticks(np.arange(round(min(lotinfoWD['AvailableLots']-10)/10)*10, max(lotinfoWD['AvailableLots'])+11, 10))
        plt.xticks(np.arange(min(lotinfo['Time']/60), max(lotinfo['Time']/60)+1, 1))

        #Do polynomial regression

        linreg = LinearRegression()

        y = pd.DataFrame(lotinfo['AvailableLots'])
        x = pd.DataFrame(lotinfo['Time'])
        poly_reg = PolynomialFeatures(degree = 4)
        x_poly = poly_reg.fit_transform(x)
        poly_reg.fit(x_poly, y)
        linreg.fit(x_poly, y)
        
        time=[6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]
        rows=[]
        for a in time:
            predtime = [[a*60]]
            pred = pd.DataFrame(predtime, columns = ['Time'])
            pred_poly = poly_reg.fit_transform(pred)
            for i in linreg.predict(pred_poly):
                for j in i:
                    j = round(j)
                    if j<0:
                        j=0
                    rows.append([a,j])
        carparkslots=pd.DataFrame(rows, columns = ['Time','Predicted carpark slots'])
        f = plt.figure(figsize=(30, 15))
        carparkslots.plot.bar(x="Time",y="Predicted carpark slots",rot=0)
        
        #Predict
        predtime = [[predhr*60]]
        pred = pd.DataFrame(predtime, columns = ['Time'])
        pred_poly = poly_reg.fit_transform(pred)
        for i in linreg.predict(pred_poly):
            for j in i:
                j = round(j)
        if (j<0):
            j = 0;
        development = carparkInfoDict["26/3/2021"].loc[carparkInfoDict["26/3/2021"]['CarParkID'] == carparkID].Development.iloc[0]

        agency = carparkInfoDict["26/3/2021"].loc[carparkInfoDict["26/3/2021"]['CarParkID'] == carparkID].Agency.iloc[0]
        if agency == "HDB":
            within_central_area = ["ACB", "BBB", "BRB1", "CY", "DUXM", "HLM", "KAB", "KAM", "KAS", "PRM", "SLS", "SR1", "SR2", "TPM", "UCS", "WCB"]
            if (carparkID in within_central_area) and (datetime.strptime(predDate, '%d/%m/%Y').weekday() == 6) and (int(datetime.strftime(predhr, '%H')) > 17):
                rates = "$1.20"
            else:
                rates = "$0.60"
        elif agency == "LTA":
            rates = "Meme"
            # URA
        else:
            URAdf = pd.read_csv("./URA.csv")
            if (datetime.strptime(predDate, '%d/%m/%Y').weekday() < 6):
                rates = URAdf.loc[URAdf['ppCode']==carparkID]["weekdayRate"].values[0]
            else:
                rates = URAdf.loc[URAdf['ppCode']==carparkID]["sunPHRate"].values[0]

        counter = counter + 1
        print(str(counter)+". "+str(j)+" carpark slots in "+development+" at "+str(predhr)+" 00 HRS. Rates: "+rates+" per half an hr")'''

'#carparkID is a list of strings, date is in the format dd/mm/yyyy , predhr is an int of hour in 24h format\ndef predictCarparkSlots(carparkIDs, predDate, predhr): \n    #Pandas, Json and formatting stuff\n    import pandas as pd\n    from datetime import datetime\n    import numpy as np\n\n    #For file iteration\n    import os\n    \n    #For Graphing\n    import seaborn as sb\n    import matplotlib.pyplot as plt \n    sb.set()\n    \n    #linreg\n    from sklearn.linear_model import LinearRegression\n    from sklearn.preprocessing import PolynomialFeatures\n\n    counter = 0\n    #Create Dictionary to store all data\n    #Dict key = Date\n    carparkInfoDict = {}\n\n    for filename in os.listdir("./Data/"):\n        if filename.endswith(".csv"): \n            #print(os.path.join("./Data/", filename))\n            path = os.path.join("./Data/", filename)\n            formatDate = filename.replace("_carpark_availability.csv", "")\n            formatDate = formatDate.replace("_", "/")